In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd

# Veri setinizi yükleyin (varsayılan olarak 'doğru_haberler' ve 'yanlis_haberler' olarak adlandırıldı)
doğru_haberler = pd.read_csv("true.csv")
yanlis_haberler = pd.read_csv("fake.csv")

# Veri önişleme
# Metin verilerini ve etiketleri birleştirin (doğru = 1, yanlış = 0)
metinler = np.concatenate([doğru_haberler['text'], yanlis_haberler['text']])
etiketler = np.concatenate([np.ones(len(doğru_haberler)), np.zeros(len(yanlis_haberler))])

# Tokenizasyon
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(metinler)
sequences = tokenizer.texts_to_sequences(metinler)
data = pad_sequences(sequences, maxlen=200)

# Model oluşturma
model = Sequential()
model.add(Embedding(5000, 100, input_length=200))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define the early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',     # Monitor the validation loss
    patience=3,             # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

# Fit the model with the early stopping callback
history = model.fit(
    data,
    etiketler,
    epochs=5,
    batch_size=64,
    validation_split=0.1,
    callbacks=[early_stopping]  # Add the callback to the training process
)

2024-01-21 14:38:16.700367: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-21 14:38:16.742424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-21 14:38:17.344991: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-01-21 14:38:38.097783: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

Epoch 1/5
632/632 [==============================] - 103s 160ms/step - loss: 0.1078 - accuracy: 0.9633 - val_loss: 0.1793 - val_accuracy: 0.9423
Epoch 2/5
632/632 [==============================] - 102s 161ms/step - loss: 0.0396 - accuracy: 0.9888 - val_loss: 0.3778 - val_accuracy: 0.8987
Epoch 3/5
632/632 [==============================] - 106s 167ms/step - loss: 0.0355 - accuracy: 0.9897 - val_loss: 0.4581 - val_accuracy: 0.9038
Epoch 4/5
632/632 [==============================] - 106s 167ms/step - loss: 0.0377 - accuracy: 0.9886 - val_loss: 0.3707 - val_accuracy: 0.8931


In [2]:
# CSV dosyasını okuyun
fake_news_df = pd.read_csv("fake.csv")

# Gerekirse, veri temizleme veya ön işleme yapın
# Örneğin, boş metinleri kaldırabilir veya tüm metinleri küçük harfe dönüştürebilirsiniz

# 'title', 'text', 'subject', 'date' sütunlarını birleştirin
# Her bir haber için birleştirilmiş metin dizesi oluşturun
# Bunun için daha karmaşık veya basit birleştirmeler yapabilirsiniz, burada basit bir örnek verilmiştir
fake_news_df['combined_text'] = fake_news_df['text'] + " " + fake_news_df['subject']

# Birleştirilmiş metinleri bir metin dosyasına yazın
with open('sahte_haberler.txt', 'w', encoding='utf-8') as f:
    for text in fake_news_df['combined_text']:
        f.write(text + "\n")


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Model ve tokenizer yükleyin
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Sahte haber veri setinizi yükleyin ve bir TextDataset oluşturun
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="sahte_haberler.txt",  # Sahte haber veri setinizin yolu
    block_size=128
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Eğitim ayarlarını yapılandırın
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",  # Model çıktılarının ve checkpoint'lerin kaydedileceği dizin
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Trainer oluşturun ve eğitimi başlatın
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

trainer.train()

# Eğitilen modeli kullanarak sahte haber metni üretin
prompt = "Write false news about this subject: economy"

inputs = tokenizer.encode(prompt, return_tensors='pt')
outputs = model.generate(inputs, max_length=200, num_return_sequences=1)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text)


Evet, verdiğiniz kod parçası, bir GPT-2 modelinin ince ayar (fine-tuning) işlemini gerçekleştirmek için kullanılır. Bu işlem sırasında, önceden eğitilmiş bir model (GPT2LMHeadModel.from_pretrained('gpt2')) alınır ve belirli bir veri seti üzerinde daha fazla eğitilerek spesifik bir göreve uyarlanır. Bu durumda, model sahte haber metinlerini içeren sahte_haberler.txt dosyası kullanılarak ince ayar yapılıyor.

İşte her adımda ne yapıldığını açıklayan bir özet:

Model ve Tokenizer Yükleme: GPT-2 modeli ve tokenizer'ı transformers kütüphanesi kullanılarak yüklenir. Tokenizer, metinleri modele verilecek token dizilerine çevirir.

TextDataset Oluşturma: TextDataset, eğitim için kullanılacak veri setini yükler ve modelin eğitileceği bloklara ayırır.

DataCollator Oluşturma: DataCollatorForLanguageModeling, token dizilerini toplu işlem haline getirir ve gerekirse padding yapar.

Eğitim Argümanlarını Yapılandırma: TrainingArguments ile eğitim parametreleri belirlenir, örneğin eğitim süresi, batch boyutu ve checkpoint kaydetme ayarları.

Trainer Oluşturma ve Eğitim: Trainer sınıfı, model eğitim sürecini yönetir ve train() metodu ile eğitimi başlatır.

Metin Üretme: Eğitim tamamlandıktan sonra, model bir prompt alır ve yeni metin dizileri üretir.

Verilen kodda trainer.train() komutu ile modelin eğitimi başlatılır ve bu işlem modeli ince ayarlamak anlamına gelir. Ancak, kodu çalıştırmadan önce sahte_haberler.txt dosyasının doğru formatlandığından ve ilgili veri setini içerdiğinden emin olmalısınız. Bu kod parçasının başarıyla çalışabilmesi için dosyanın ve ortamın düzgün şekilde hazırlanmış olması gerekmektedir.

Ayrıca, bu kodun çalıştırılması GPU veya TPU gibi yüksek işlem gücüne sahip bir makine gerektirebilir, çünkü GPT-2 gibi büyük modellerin eğitimi ciddi miktarda hesaplama kaynağı tüketir. Eğer hali hazırda bir Hugging Face hesabınız varsa ve uygun API token'ınıza sahipseniz, bu işlemi Hugging Face'in sunucularında gerçekleştirebilirsiniz.

Eğitim sürecinin tamamlanmasının ardından, modeliniz özel veri setinize göre ince ayarlanmış olacak ve bu veri setinin stilinde metin üretebilecektir.





